## Preparación y Unificación de Datasets de Salarios
### 1. Carga y Limpieza Inicial de los Datasets

Cargamos los datasets y aplicamos la limpieza básica identificada en el EDA previo.

In [60]:
%pip install pycountry

Note: you may need to restart the kernel to use updated packages.


In [61]:
import pandas as pd
import numpy as np
import pycountry

try:
    df1_original = pd.read_csv("./data_source/ds_salaries.csv")
    df2_original = pd.read_csv("./data_source/jobs_in_data.csv")
except FileNotFoundError:
    print("Error: Uno o ambos archivos CSV no se encontraron. Verifica las rutas: ./data_source/")
    # Crear dataframes vacíos para evitar errores si los archivos no se encuentran
    df1_original = pd.DataFrame()
    df2_original = pd.DataFrame()

df1 = df1_original.copy()
df2 = df2_original.copy()

# Limpieza Dataset 1 (ds_salaries.csv)
if 'Unnamed: 0' in df1.columns:
    df1.drop(['Unnamed: 0'], axis=1, inplace=True)
df1.drop_duplicates(inplace=True)
print(f"Forma de Dataset 1 después de limpieza inicial: {df1.shape}")

# Limpieza Dataset 2 (jobs_in_data.csv)
if 'job_category' in df2.columns:  # Esta columna no existe en ds_salaries
    df2.drop(['job_category'], axis=1, inplace=True)
df2.drop_duplicates(inplace=True)
print(f"Forma de Dataset 2 después de limpieza inicial: {df2.shape}")



Forma de Dataset 1 después de limpieza inicial: (565, 11)
Forma de Dataset 2 después de limpieza inicial: (5341, 11)


### 2. Estandarización de Columnas Comunes

Ahora, unificaremos los valores de las columnas que tienen diferentes formatos.

#### 2.1. `experience_level`

In [62]:
if not df1.empty:
    # Mapeo para dataset1
    exp_level_map_df1 = {
        'EN': 'Entry-level',
        'MI': 'Mid-level',
        'SE': 'Senior',
        'EX': 'Executive'
    }
    df1['experience_level'] = df1['experience_level'].map(exp_level_map_df1).fillna(df1['experience_level'])
    print("Valores únicos de 'experience_level' en df1 después de mapeo:")
    print(df1['experience_level'].value_counts())

if not df2.empty:
    # Para df2, los valores ya deberían ser descriptivos. Solo mostramos para confirmar.
    print("\nValores únicos de 'experience_level' en df2 (sin cambios esperados):")
    print(df2['experience_level'].value_counts())


Valores únicos de 'experience_level' en df1 después de mapeo:
experience_level
Senior         243
Mid-level      208
Entry-level     88
Executive       26
Name: count, dtype: int64

Valores únicos de 'experience_level' en df2 (sin cambios esperados):
experience_level
Senior         3444
Mid-level      1274
Entry-level     400
Executive       223
Name: count, dtype: int64


#### 2.2. `employment_type`

In [63]:
if not df1.empty:
    # Mapeo para dataset1
    emp_type_map_df1 = {
        'PT': 'Part-time',
        'FT': 'Full-time',
        'CT': 'Contract',
        'FL': 'Freelance'
    }
    df1['employment_type'] = df1['employment_type'].map(emp_type_map_df1).fillna(df1['employment_type'])
    print("Valores únicos de 'employment_type' en df1 después de mapeo:")
    print(df1['employment_type'].value_counts())

if not df2.empty:
    # Para df2, asumimos que los valores ya son descriptivos (ej. 'Full-time').
    # Si hubiera variaciones como 'Full Time', se necesitaría un mapeo adicional aquí.
    print("\nValores únicos de 'employment_type' en df2 (sin cambios esperados):")
    print(df2['employment_type'].value_counts())


Valores únicos de 'employment_type' en df1 después de mapeo:
employment_type
Full-time    546
Part-time     10
Contract       5
Freelance      4
Name: count, dtype: int64

Valores únicos de 'employment_type' en df2 (sin cambios esperados):
employment_type
Full-time    5296
Contract       19
Part-time      15
Freelance      11
Name: count, dtype: int64


#### 2.3. `employee_residence` y `company_location`

Convertiremos los códigos de país de `dataset1` a nombres completos usando `pycountry`. `dataset2` ya tiene nombres completos.

In [64]:
def code_to_country_name(code):
    if not pycountry or pd.isna(code) or not isinstance(code, str) or len(code) != 2:
        return code  # Devuelve el código original si no es un código válido o pycountry no está disponible
    try:
        country = pycountry.countries.get(alpha_2=code)
        return country.name if country else code
    except Exception:
        return code  # En caso de error, devuelve el código original

if not df1.empty and pycountry is not None:
    print("\nTransformando códigos de país a nombres en df1...")
    df1['employee_residence'] = df1['employee_residence'].apply(code_to_country_name)
    df1['company_location'] = df1['company_location'].apply(code_to_country_name)
    print("Primeros ejemplos de 'employee_residence' en df1 después de transformación:")
    print(df1['employee_residence'].head())
    print("Primeros ejemplos de 'company_location' en df1 después de transformación:")
    print(df1['company_location'].head())
elif pycountry is None:
    print("\npycountry no está disponible, no se transformarán los códigos de país en df1.")

if not df2.empty:
    # Solo mostramos ejemplos de df2, ya que se asume que tienen nombres completos.
    print("\nPrimeros ejemplos de 'employee_residence' en df2 (sin cambios esperados):")
    print(df2['employee_residence'].head())
    print("Primeros ejemplos de 'company_location' en df2 (sin cambios esperados):")
    print(df2['company_location'].head())



Transformando códigos de país a nombres en df1...
Primeros ejemplos de 'employee_residence' en df1 después de transformación:
0           Germany
1             Japan
2    United Kingdom
3          Honduras
4     United States
Name: employee_residence, dtype: object
Primeros ejemplos de 'company_location' en df1 después de transformación:
0           Germany
1             Japan
2    United Kingdom
3          Honduras
4     United States
Name: company_location, dtype: object

Primeros ejemplos de 'employee_residence' en df2 (sin cambios esperados):
0          Germany
1    United States
2    United States
3    United States
4    United States
Name: employee_residence, dtype: object
Primeros ejemplos de 'company_location' en df2 (sin cambios esperados):
0          Germany
1    United States
2    United States
3    United States
4    United States
Name: company_location, dtype: object


#### 2.4. `remote_ratio` (dataset1) y `work_setting` (dataset2)

Unificaremos esto en una única columna llamada `work_setting`.

In [65]:
if not df1.empty and 'remote_ratio' in df1.columns:
    # Mapeo para dataset1
    remote_ratio_map_df1 = {
        0: 'In-person',
        50: 'Hybrid',
        100: 'Remote'
    }
    df1['work_setting'] = df1['remote_ratio'].map(remote_ratio_map_df1).fillna(df1['remote_ratio'])
    df1.drop('remote_ratio', axis=1, inplace=True)
    print("Valores únicos de 'work_setting' (originada de 'remote_ratio') en df1 después de mapeo:")
    print(df1['work_setting'].value_counts())

if not df2.empty and 'work_setting' in df2.columns:
    # Para df2, los valores ya deberían ser descriptivos. Solo mostramos para confirmar.
    print("\nValores únicos de 'work_setting' en df2 (sin cambios esperados):")
    print(df2['work_setting'].value_counts())


Valores únicos de 'work_setting' (originada de 'remote_ratio') en df1 después de mapeo:
work_setting
Remote       346
In-person    121
Hybrid        98
Name: count, dtype: int64

Valores únicos de 'work_setting' en df2 (sin cambios esperados):
work_setting
In-person    2913
Remote       2239
Hybrid        189
Name: count, dtype: int64


### 3. Verificación de Columnas y Concatenación


In [66]:
if not df1.empty and not df2.empty:
    # Columnas esperadas en el dataset unificado
    expected_columns = [
        'work_year', 'experience_level', 'employment_type', 'job_title',
        'salary', 'salary_currency', 'salary_in_usd', 'employee_residence',
        'company_location', 'company_size', 'work_setting'
    ]
    
    # Filtrar columnas según la lista esperada
    df1_cols_to_keep = [col for col in expected_columns if col in df1.columns]
    df2_cols_to_keep = [col for col in expected_columns if col in df2.columns]
    
    df1_aligned = df1[df1_cols_to_keep]
    df2_aligned = df2[df2_cols_to_keep]
    
    # Verificar alineación exacta de columnas
    if list(df1_aligned.columns) == list(df2_aligned.columns):
        print("\nLas columnas están alineadas. Procediendo a la concatenación.")
        df_merged = pd.concat([df1_aligned, df2_aligned], ignore_index=True)
        print(f"Forma del dataset unificado: {df_merged.shape}")
        
        # Eliminar duplicados
        print(f"Duplicados en el dataset unificado: {df_merged.duplicated().sum()}")
        df_merged.drop_duplicates(inplace=True)
        print(f"Forma del dataset unificado después de eliminar duplicados: {df_merged.shape}")
        
        print("\nInformación del dataset unificado:")
        df_merged.info()
        
        print("\nPrimeras filas del dataset unificado:")
        print(df_merged.head())
        
        print("\nÚltimas filas del dataset unificado:")
        print(df_merged.tail())
    else:
        print("\nError: Las columnas no pudieron ser alineadas completamente.")
        print(f"Columnas df1: {list(df1_aligned.columns)}")
        print(f"Columnas df2: {list(df2_aligned.columns)}")
        df_merged = pd.DataFrame()  # Evitar errores posteriores
elif df1.empty or df2.empty:
    print("Uno de los dataframes está vacío. No se puede realizar la unificación.")
    df_merged = pd.DataFrame()



Las columnas están alineadas. Procediendo a la concatenación.
Forma del dataset unificado: (5906, 11)
Duplicados en el dataset unificado: 429
Forma del dataset unificado después de eliminar duplicados: (5477, 11)

Información del dataset unificado:
<class 'pandas.core.frame.DataFrame'>
Index: 5477 entries, 0 to 5876
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           5477 non-null   int64 
 1   experience_level    5477 non-null   object
 2   employment_type     5477 non-null   object
 3   job_title           5477 non-null   object
 4   salary              5477 non-null   int64 
 5   salary_currency     5477 non-null   object
 6   salary_in_usd       5477 non-null   int64 
 7   employee_residence  5477 non-null   object
 8   company_location    5477 non-null   object
 9   company_size        5477 non-null   object
 10  work_setting        5477 non-null   object
dtypes: int64(3), object

### 4. Verificación Final de Columnas Unificadas


In [67]:
# %%
if not df_merged.empty:
    columns_to_check = ['experience_level', 'employment_type', 'employee_residence', 'company_location', 'work_setting']
    for col in columns_to_check:
        if col in df_merged.columns:
            print(f"\nValores únicos y conteos para '{col}' en el dataset unificado:")
            print(df_merged[col].value_counts().head(10))  # Mostrar top 10 para brevedad
        else:
            print(f"La columna '{col}' no se encuentra en el dataset unificado.")



Valores únicos y conteos para 'experience_level' en el dataset unificado:
experience_level
Senior         3468
Mid-level      1352
Entry-level     427
Executive       230
Name: count, dtype: int64

Valores únicos y conteos para 'employment_type' en el dataset unificado:
employment_type
Full-time    5424
Contract       20
Part-time      20
Freelance      13
Name: count, dtype: int64

Valores únicos y conteos para 'employee_residence' en el dataset unificado:
employee_residence
United States     4262
United Kingdom     380
Canada             203
Spain               71
Germany             69
France              56
India               41
Portugal            29
Netherlands         22
Australia           21
Name: count, dtype: int64

Valores únicos y conteos para 'company_location' en el dataset unificado:
company_location
United States     4317
United Kingdom     388
Canada             205
Germany             77
Spain               67
France              51
India               30
Portugal 

### 5. Mapeo de variables categoricas con Ordinal Encoding

In [68]:
unique_job_titles = df_merged['job_title'].unique()
job_title_mapping = {title: idx for idx, title in enumerate(sorted(unique_job_titles))}

df_merged['job_title'] = df_merged['job_title'].map(job_title_mapping)

unique_salary_currency = df_merged['salary_currency'].unique()
job_salary_currency_mapping = {title: idx for idx, title in enumerate(sorted(unique_salary_currency))}

df_merged['salary_currency'] = df_merged['salary_currency'].map(job_salary_currency_mapping)

unique_employee_residence = df_merged['employee_residence'].unique()
employee_residence_mapping = {title: idx for idx, title in enumerate(sorted(unique_employee_residence))}

df_merged['employee_residence'] = df_merged['employee_residence'].map(employee_residence_mapping)

unique_company_location = df_merged['company_location'].unique()
company_location_mapping = {title: idx for idx, title in enumerate(sorted(unique_company_location))}

df_merged['company_location'] = df_merged['company_location'].map(company_location_mapping)

unique_salary_currency = df_merged['salary_currency'].unique()
job_salary_currency_mapping = {title: idx for idx, title in enumerate(sorted(unique_salary_currency))}

df_merged['salary_currency'] = df_merged['salary_currency'].map(job_salary_currency_mapping)

experience_level_mapping = {
    'Entry-level': 0,
    'Mid-level': 1,
    'Senior': 2,
    'Executive': 3
}
df_merged["experience_level"] = df_merged["experience_level"].map(experience_level_mapping)

company_size_mapping = {
    'S': 0,
    'M': 1,
    'L': 2
}
df_merged["company_size"] = df_merged["company_size"].map(company_size_mapping)

employment_type_mapping = {
    'Full-time': 0,
    'Contract': 1,
    'Freelance': 2,
    'Part-time': 3
}

df_merged["employment_type"] = df_merged["employment_type"].map(employment_type_mapping)

work_setting_mapping = {
    'Hybrid': 0,
    'In-person': 1,
    'Remote': 2,
}

df_merged["work_setting"] = df_merged["work_setting"].map(work_setting_mapping)

# Verificar los cambios
df_merged.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,company_location,company_size,work_setting
0,2020,1,0,71,70000,7,79833,31,25,2,1
1,2020,2,0,106,260000,16,260000,44,39,0,1
2,2020,2,0,21,85000,8,109024,86,75,1,0
3,2020,1,0,119,20000,16,20000,34,29,0,1
4,2020,2,0,99,150000,16,150000,87,76,2,0


### 6. Guardar el Dataset Unificado

In [69]:
if not df_merged.empty:
    try:
        output_path = "./data_source/salaries_data_unified.csv"
        df_merged.to_csv(output_path, index=False)
        print(f"\nDataset unificado guardado exitosamente en: {output_path}")
    except Exception as e:
        print(f"\nError al guardar el dataset unificado: {e}")
else:
    print("\nEl dataset unificado está vacío, no se guardará ningún archivo.")



Dataset unificado guardado exitosamente en: ./data_source/salaries_data_unified.csv
